## DAT405 Introduction to Data Science and AI
# Assignment 4: Spam classification using Naïve Bayes

Student name | Hours spent on the tasks
------------ | -------------
Lenia Malki | 10
Maële Belmont | 10

## Setup
Python modules need to be loaded to solve the tasks.

In [4]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

## Introduction
There will be an overall grade for this assignment. To get a pass grade (grade 3), you need to pass 
items 1-3 below. To receive higher grades, finish items 4 and 5 as well.<br>
In this assignment you will implement a Naïve Bayes classifier in Python that will classify emails into 
spam and non-spam (“ham”) classes. Your program should be able to train on a given set of spam 
and “ham” datasets.   You will work with the datasets available at https://spamassassin.apache.org/old/publiccorpus/. 
There are three types of files in this location: 
1. easy-ham: non-spam messages typically quite easy to differentiate from spam messages. 
2. hard-ham: non-spam messages more difficult to differentiate
3. spam: spam messages

Read the “readme.html” for a full description of the file contents. The .bz2-file can be unzipped using 
the linux command
tar –xjvf file.bz2
which will result in a directory named “file”, containing all email messages as separate files. 

## Task 1 - Preprocessing: 
#### a. Note that the email files contain a lot of extra information, besides the actual message. Ignore that for now and run on the entire text. Further down (in the higher grade part), you will be asked to filter out the headers and footers. 

#### b. We don’t want to train and test on the same data. Split the spam and the ham datasets in a training set and a test set.

## Task 2 - Write a Python program that:
#### a. Uses four datasets (hamtrain, spamtrain, hamtest, and spamtest)

#### b. Using a Naïve Bayes classifier (e.g. Sklearn), classifies the test sets and reports the  percentage of ham and spam test sets that were classified correctly. You can use CountVectorizer to transform the email texts into vectors. Please note that there are different types of Naïve Bayes Classifier in SKlearn (Document is available here). Test two of these classifiers: 1. Multinomial Naive Bayes and 2. Bernoulli Naive Bayes that are well suited for this problem. For the case of Bernoulli Naive Bayes you should use the parameter binarize to make the features binary. Discuss the differences between these two classifiers. 

## Task 3 - Run your program on
#### i. Spam versus easy-ham

#### ii. Spam versus hard-ham

## Task 4 - To avoid classification based on common and uninformative words it is common to filter these out. 
#### a. Argue why this may be useful. Try finding the words that are too common/uncommon in the dataset. 

#### b. Use the parameters in Sklearn’s CountVectorizer to filter out these words. Run the updated program on your data and record how the results differ from 3. You have two options to do this in Sklearn: either using the words found in part (a) or letting Sklearn do it for you.

## Task 5 - Filter out the headers and the footers of the emails before you run on them. The format may vary somewhat between emails, which can make this a bit tricky, so perfect filtering is not required. Run your program again and answer the following questions: 
#### a. Does the result improve from 3 and 4?

#### b. The split of the data set into a training set and a test set can lead to very skewed results. Why is this, and do you have suggestions on remedies?

#### c. What do you expect would happen if your training set were mostly spam messages while your test set were mostly ham messages?